In [4]:
import numpy as np
import pandas as pd
import openpyxl

df=pd.read_excel('Telco_customer_churn.xlsx')
df_raw=pd.read_excel('Telco_customer_churn.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

In [5]:
df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors='coerce') ##coerce changes the non convvertable values such as alphabets and missing values to numeric data type
df['Total Charges']=df['Total Charges'].fillna(df['Total Charges'].median())
df['Total Charges'].isnull().sum()

np.int64(0)

In [6]:
# check churn distribution (use numeric churn if present)
print(df['Churn Value'].value_counts())
print(df['Churn Value'].value_counts(normalize=True)) ##gives percentage of each category

Churn Value
0    5174
1    1869
Name: count, dtype: int64
Churn Value
0    0.73463
1    0.26537
Name: proportion, dtype: float64


In [7]:
# normalize column names: strip, lower, replace spaces with _
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df.columns.tolist()

['customerid',
 'count',
 'country',
 'state',
 'city',
 'zip_code',
 'lat_long',
 'latitude',
 'longitude',
 'gender',
 'senior_citizen',
 'partner',
 'dependents',
 'tenure_months',
 'phone_service',
 'multiple_lines',
 'internet_service',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'contract',
 'paperless_billing',
 'payment_method',
 'monthly_charges',
 'total_charges',
 'churn_label',
 'churn_value',
 'churn_score',
 'cltv',
 'churn_reason']

In [8]:
drop_cols = [
    'churn_reason', 'churn_score', 'cltv',
    'country','state','city','zip_code',
    'lat_long','latitude','longitude'
]

# drop safely (errors='ignore' if some names differ)
df.drop(columns=drop_cols, inplace=True, errors='ignore')


If any column(s) in drop_cols are not present in df, Pandas will skip them silently instead of raising a KeyError.

If you used errors='raise' (default), trying to drop a non-existent column would raise an exception.

In [9]:
df.duplicated(subset='customerid').sum()   # if >0 investigate

### df = df.drop_duplicates(subset='customerid', keep='first') keep first occurence
### df = df.drop_duplicates(subset='customerid', keep='last') keep last occurence
### df = df.drop_duplicates(subset='customerid', keep=False) remove all duplicates entirely
# optionally reset index
### df = df.reset_index(drop=True)

np.int64(0)

In [10]:
# numeric summary
print(df.describe().T)

# categorical columns quick counts
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
for c in cat_cols:
    print(c, ':', df[c].nunique())
    print(df[c].value_counts().head(5))
    print('---')

                  count         mean          std    min      25%       50%  \
count            7043.0     1.000000     0.000000   1.00    1.000     1.000   
tenure_months    7043.0    32.371149    24.559481   0.00    9.000    29.000   
monthly_charges  7043.0    64.761692    30.090047  18.25   35.500    70.350   
total_charges    7043.0  2281.916928  2265.270398  18.80  402.225  1397.475   
churn_value      7043.0     0.265370     0.441561   0.00    0.000     0.000   

                     75%      max  
count               1.00     1.00  
tenure_months      55.00    72.00  
monthly_charges    89.85   118.75  
total_charges    3786.60  8684.80  
churn_value         1.00     1.00  
customerid : 7043
customerid
3186-AJIEK    1
3668-QPYBK    1
9237-HQITU    1
7203-OYKCT    1
9837-FWLCH    1
Name: count, dtype: int64
---
gender : 2
gender
Male      3555
Female    3488
Name: count, dtype: int64
---
senior_citizen : 2
senior_citizen
No     5901
Yes    1142
Name: count, dtype: int64
---
part

In [11]:
## replace the no internet service and no phone service as no and also changing Yes, YES to yes.
replace_map = {
    'no internet service': 'no',
    'no phone service': 'no'
}

for c in cat_cols:
    if df[c].dtype == 'object':
        df[c] = df[c].str.lower().replace(replace_map)


In [12]:
# rename numeric columns after lowercasing earlier
# make tenure buckets
df['tenure_months'] = df.get('tenure_months')  # ensure name presence
# if original was tenure_months; if not check df.columns
# create tenure_group (adjust bins if needed)
df['tenure_group'] = pd.cut(df['tenure_months'],
                            bins=[0,3,6,12,24,48,120],
                            labels=['0-3','3-6','6-12','12-24','24-48','48+'],
                            right=False)

# average charge per month guard for zero tenure
df['avg_charge'] = df['total_charges'] / df['tenure_months'].replace({0:1})


In [13]:
# use churn_value (numeric) if present
y = df['churn_value']

# drop id + label from features
X = df.drop(columns=['customerid','churn_label','churn_value'], errors='ignore')
X.shape, y.shape


((7043, 22), (7043,))

In [14]:
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
cat_cols.append('tenure_group')

# show
print('num:', num_cols)
print('cat:', cat_cols)


num: ['count', 'tenure_months', 'monthly_charges', 'total_charges', 'avg_charge']
cat: ['gender', 'senior_citizen', 'partner', 'dependents', 'phone_service', 'multiple_lines', 'internet_service', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'contract', 'paperless_billing', 'payment_method', 'tenure_group']


In [15]:
## Quick train/test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [16]:
## Preprocessing (same for all models)

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)
    ],
    remainder='passthrough'
)


In [17]:
## Logistic Regression Piepline

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

log_reg = Pipeline(steps=[
    ('prep', preprocessor),
    ('clf', LogisticRegression(
        max_iter=6000,
        class_weight='balanced',
        solver='lbfgs'
    ))
])

# Train
log_reg.fit(X_train, y_train)

# Predict
y_pred_lr = log_reg.predict(X_test)
y_proba_lr = log_reg.predict_proba(X_test)[:,1]

In [18]:
## Random Forest Pipeline

from sklearn.ensemble import RandomForestClassifier

rf = Pipeline(steps=[
    ('prep', preprocessor),
    ('clf', RandomForestClassifier(
        n_estimators=300,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ))
])

# Train
rf.fit(X_train, y_train)

# Predict
y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:,1]


In [23]:
## XGBoost Pipeline

from xgboost import XGBClassifier

xgb = Pipeline(steps=[
    ('prep', preprocessor),
    ('clf', XGBClassifier(
        eval_metric='logloss',
        n_estimators=300,
        learning_rate=0.5,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=1    # tune for imbalance later
    ))
])

# Train
xgb.fit(X_train, y_train)

# Predict
y_pred_xgb = xgb.predict(X_test)
y_proba_xgb = xgb.predict_proba(X_test)[:,1]


In [24]:
from sklearn.metrics import classification_report, roc_auc_score

def evaluate_model(name, y_test, y_pred, y_proba):
    print(f"\n====== {name} ======")
    print(classification_report(y_test, y_pred))
    print("ROC-AUC:", roc_auc_score(y_test, y_proba))

In [25]:
evaluate_model("Logistic Regression", y_test, y_pred_lr, y_proba_lr)
evaluate_model("Random Forest", y_test, y_pred_rf, y_proba_rf)
evaluate_model("XGBoost", y_test, y_pred_xgb, y_proba_xgb)



====== Logistic Regression ======
              precision    recall  f1-score   support

           0       0.91      0.73      0.81      1035
           1       0.52      0.79      0.62       374

    accuracy                           0.75      1409
   macro avg       0.71      0.76      0.72      1409
weighted avg       0.80      0.75      0.76      1409

ROC-AUC: 0.8552352166162909

====== Random Forest ======
              precision    recall  f1-score   support

           0       0.83      0.90      0.86      1035
           1       0.64      0.50      0.56       374

    accuracy                           0.79      1409
   macro avg       0.74      0.70      0.71      1409
weighted avg       0.78      0.79      0.78      1409

ROC-AUC: 0.8391821023534579

====== XGBoost ======
              precision    recall  f1-score   support

           0       0.84      0.87      0.85      1035
           1       0.59      0.53      0.56       374

    accuracy                           

#### ROC-AUC
“AUC measures how well the model ranks positive examples above negative ones. An AUC of 0.85 means that if I randomly pick a churner and a non-churner, the model assigns a higher probability to the churner 85% of the time. It is independent of threshold and shows how well the model separates the two classes.”  
ROC-AUC measures ranking ability, not classification accuracy.  
“Among all possible churners vs non-churners, how often does the model assign a higher score to the churner?”  
This means we only care about the probabilities, not the threshold.  
  
“ROC-AUC is a single, global metric evaluated over the entire dataset (usually the test set). It is not calculated per customer. Each customer gets a probability score, but ROC-AUC measures how well the model ranks churners above non-churners across the whole population.”